# Rush Hour

## Un modèle pour RushHour

### Un exemple d'utilisation programmatique

In [1]:
from RushHour import *

In [2]:
plateau = Plateau(['A2R00','X2R21','C2R44','R3R52','O3D05','P3D10','Q3D13','B2D40']); plateau

+------+
|AA   O|
|P  Q O|
|PXXQ O 
|P  Q  |
|B   CC|
|B RRR |
+------+

In [3]:
plateau.recule("Q")

+------+
|AA Q O|
|P  Q O|
|PXXQ O 
|P     |
|B   CC|
|B RRR |
+------+

In [4]:
plateau.est_gagnant()

False

Chargement du 40ème défi de RushHour (voir les fichiers dans <a href="RushHourDefis/">RushHourDefis/</a>):

In [5]:
Plateau(40)

+------+
|OAA B |
|OCD BP|
|OCDXXP 
|QQQE P|
|  FEKK|
|HHFII |
+------+

### Un exemple d'interface utilisateur minimale

In [6]:
from ipywidgets import interact_manual
from RushHour import Plateau
plateau = Plateau(['A2R00','X2R21','C2R44','R3R52','O3D05','P3D10','Q3D13','B2D40']); plateau
@interact_manual
def step(voiture=plateau.voitures.keys(), distance=[0,-1,1]):
    global plateau
    plateau2 = plateau.avance(voiture, distance)
    if plateau2:
        plateau = plateau2
    return plateau

interactive(children=(Dropdown(description='voiture', options=('A', 'X', 'C', 'R', 'O', 'P', 'Q', 'B'), value='A'), Dropdown(description='distance', options=(0, -1, 1), value=0), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

## Une application graphique basée sur les widgets de Jupyter

In [7]:
from ipywidgets import Button, Dropdown, VBox, HBox, Textarea
from bqplot import Figure, LinearScale, Lines

In [18]:
class Application:
    def __init__(self):
        self.niveau  = Dropdown(options=[('Niveau {}'.format(i), i) for i in RushHour.niveaux()])
        self.voiture = Dropdown(options=[])
        self.modele = Plateau(1)
        self.vue = Figure(scale_x=LinearScale(min=0, max=self.modele.dimension),
                          scale_y=LinearScale(min=self.modele.dimension, max=0)
                         )
        self.vue.layout.width="75ex"
        self.vue.layout.width="75ex"
        self.vue.layout.height=self.vue.layout.width;
        self.niveau.observe(lambda widget: self.change_niveau(widget.owner.value))
        self.widget = VBox([self.niveau,
                            self.vue,
                            self.boutton_direction('U'),
                            HBox([self.boutton_direction('L'), self.voiture, self.boutton_direction('R')]),
                            self.boutton_direction('D')
                           ])
        self.widget.layout.align_items = 'center'
        self.change_niveau(1)

    # Call backs
    def change_niveau(self, i):
        self.modele = Plateau(i)
        self.voiture.options = self.modele.voitures.keys()
        self.mise_a_jour_vue()
        
    def on_click_direction(self, boutton):
        plateau = self.modele.deplace(self.voiture.value, boutton.description)
        if plateau is not None:
            self.modele = plateau
            self.mise_a_jour_vue()
        
    def choix_voiture(self, lettre):
        self.voiture.value = lettre

    def mise_a_jour_vue(self):
        marks = []
        for lettre,voiture in self.modele.voitures.items():
            for (i,j) in voiture.cases():
                case = Lines(y=[i,i+1,i+1,i],
                             x=[j,j,j+1,j+1],
                             scales={'x':self.vue.scale_x,
                                     'y':self.vue.scale_y},
                             fill='inside',
                             colors=[couleurs[lettre]])
                case.lettre=lettre
                
                case.on_element_click(lambda case, _: self.choix_voiture(case.lettre))
                marks.append(case)
        self.vue.marks = marks

    def boutton_direction(self, direction):
        boutton = Button(description=direction)
        boutton.on_click(self.on_click_direction)
        return boutton
    
    def display(self):
        return self.widget

Application().display()

/opt/miniconda3/envs/test-widgets/lib/python3.6/site-packages/bqplot/traits.py:137: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(a.dtype, np.float):


VBox(children=(Dropdown(options=(('Niveau 1', 1), ('Niveau 3', 3), ('Niveau 7', 7), ('Niveau 8', 8), ('Niveau 11', 11), ('Niveau 15', 15), ('Niveau 21', 21), ('Niveau 38', 38), ('Niveau 40', 40)), value=1), Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='75ex', min_width='125px', width='75ex'), marks=[Lines(colors=['lightgreen'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([0, 0, 1, 1]), y=array([0, 1, 1, 0])), Lines(colors=['lightgreen'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([1, 1, 2, 2]), y=array([0, 1, 1, 0])), Lines(colors=['red'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([1, 1, 2, 2]), y=array([2, 3, 3, 2])), Lines(colors=['red'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([2, 2, 3, 3]), y=array([2, 3, 3, 2])), Lines(colors=['lightblue'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([4, 4, 5, 5]), y=array([4, 5, 5, 4])), Lines(colors=['lightblue'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([5, 5, 6, 6]), y=array([4, 5, 5, 4])), Lines(colors=['green'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([2, 2, 3, 3]), y=array([5, 6, 6, 5])), Lines(colors=['green'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([3, 3, 4, 4]), y=array([5, 6, 6, 5])), Lines(colors=['green'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([4, 4, 5, 5]), y=array([5, 6, 6, 5])), Lines(colors=['yellow'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), '

## Exercice

Déterminer une séquence minimale de coup permettant de résoudre le défi 40 de Rush Hour:

In [18]:
### BEGIN SOLUTION
solution = RushHour.solution(40)
### END SOLUTION

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [13]:
RushHour.est_solution(40, solution)

True